In [21]:
import pandas as pd
import numpy as np
import math
import warnings

warnings.filterwarnings('ignore')

In [22]:
sales_data = pd.read_csv("Sara's copy of sales.csv") #replace with file name

In [23]:
sales_data
#review data to see that it looks correct:

,Total Sales,Oct FY15,Nov FY15,Dec FY15,Jan FY15,Feb FY15,Mar FY15,Apr FY15,May FY15,Jun FY15,...,Jul-FY20,Aug-FY20,Sept-FY20,Oct-FY21,Nov-FY21,Dec-FY21,Jan-FY21,Feb-FY21,Mar-FY21,Apr-FY21
0,Carmel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,314,261,266,213,328,551,258,170,252,156
1,Fishers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,351,318,245,202,261,221,575,314,272,244
2,Greenwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,223,261,254,219,632,362,216,191,186,269
3,Noblesville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,North Indianapolis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,South Indianapolis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48,56,53,31,31,42,38,21,33,17
6,Zionsville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,80,75,46,23,20,33,15,16,35,40
7,Batavia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81,104,90,57,54,54,39,27,87,75
8,Dekalb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,155,254,135,114,98,128,104,84,96,86
9,East Moline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [24]:
num_months = sales_data.shape[1] - 1

In [25]:
a1 = 'Total Sales' #replace Total Sales with what's in the A1 cell of your Excel file


markets = sales_data[a1].unique()
markets
#check to see that only market names remain below

array(['Carmel', 'Fishers', 'Greenwood', 'Noblesville',
       'North Indianapolis', 'South Indianapolis', 'Zionsville',
       'Batavia', 'Dekalb', 'East Moline', 'Geneva', 'Genoa',
       'North Aurora', 'Oswego', 'Plainfield', 'Plano', 'Rock Island',
       'Romeoville', 'Sandwich', 'South Elgin', 'St. Charles',
       'Sugar Grove', 'Sycamore', 'Yorkville', 'Berea', 'Lexington',
       'Nicholasville', 'Richmond', 'Versailles', 'Lansing', 'Ypsilanti',
       'Austin', 'Cologne', 'Jaguar', 'Lakeville', 'Mancato',
       'Northfield', 'Rochester', 'Victoria', 'Waconia', 'Watertown',
       'Ames', 'Bettendorf', 'Clive', 'Davenport', 'Grimes', 'Johnston',
       'Le Claire', 'Nevada', 'Urbandale', 'Fayetteville', 'Greenville',
       'Hope Mills', 'Raeford', 'Spring Lake', 'Clayton', 'Englewood',
       'Huber Heights', 'Tipp City', 'Troy', 'Union', 'Vandalia',
       'West Milton', 'Tallahassee', 'Connersville', 'Crawfordsville',
       'Franklin', 'Greencastle', 'Huntington', 'Lafay

In [26]:
num_markets = len(markets)

In [27]:
new_table = pd.DataFrame(columns = ['Market'])
new_table['Market'] = ['Carmel', 'Fishers', 'Greenwood', 'Noblesville',
       'North Indianapolis', 'South Indianapolis', 'Zionsville',
       'Batavia', 'Dekalb', 'East Moline', 'Geneva', 'Genoa',
       'North Aurora', 'Oswego', 'Plainfield', 'Plano', 'Rock Island',
       'Romeoville', 'Sandwich', 'South Elgin', 'St. Charles',
       'Sugar Grove', 'Sycamore', 'Yorkville', 'Berea', 'Lexington',
       'Nicholasville', 'Richmond', 'Versailles', 'Lansing', 'Ypsilanti',
       'Austin', 'Cologne', 'Jaguar', 'Lakeville', 'Mancato',
       'Northfield', 'Rochester', 'Victoria', 'Waconia', 'Watertown',
       'Ames', 'Bettendorf', 'Clive', 'Davenport', 'Grimes', 'Johnston',
       'Le Claire', 'Nevada', 'Urbandale', 'Fayetteville', 'Greenville',
       'Hope Mills', 'Raeford', 'Spring Lake', 'Clayton', 'Englewood',
       'Huber Heights', 'Tipp City', 'Troy', 'Union', 'Vandalia',
       'West Milton', 'Tallahassee', 'Connersville', 'Crawfordsville',
       'Franklin', 'Greencastle', 'Huntington', 'Lafayette', 'Lebanon',
       'Madison', 'New Castle', 'Seymour', 'Vincennes', 'Wabash',
       'Westfield', 'BloomingtonNormal', 'Ann Arbor', 'Detroit',
       'East Lansing', 'Grand Rapids', 'Southfield', 'Toledo']

In [28]:
final_table = pd.concat([new_table]*num_months, ignore_index=True).sort_values('Market')

In [29]:
#check to see that dates below are accurate
dates = pd.DataFrame({'date': ['Oct FY15', 'Nov FY15', 'Dec FY15', 'Jan FY15', 'Feb FY15', 'Mar FY15',
       'Apr FY15', 'May FY15', 'Jun FY15', 'Jul FY15', 'Aug FY15', 'Sept FY15',
       'Oct FY16', 'Nov FY16', 'Dec FY16', 'Jan FY16', 'Feb FY16', 'Mar FY16',
       'Apr FY16', 'May FY16', 'Jun FY16', 'Jul FY16', 'Aug FY16', 'Sept FY16',
       'Oct FY17', 'Nov FY17', 'Dec FY17', 'Jan FY17', 'Feb FY17', 'Mar FY17',
       'Apr FY17', 'May FY17', 'Jun FY17', 'Jul FY17', 'Aug FY17', 'Sept FY17',
       'Oct-FY18', 'Nov-FY18', 'Dec-FY18', 'Jan-FY18', 'Feb-FY18', 'Mar-FY18',
       'Apr-FY18', 'May-FY18', 'Jun-FY18', 'Jul-FY18', 'Aug-FY18', 'Sept-FY18',
       'Oct-FY19', 'Nov-FY19', 'Dec-FY19', 'Jan-FY19', 'Feb-FY19', 'Mar-FY19',
       'Apr-FY19', 'May-FY19', 'Jun-FY19', 'Jul-FY19', 'Aug-FY19', 'Sept-FY19',
       'Oct-FY20', 'Nov-FY20', 'Dec-FY20', 'Jan-FY20', 'Feb-FY20', 'Mar-FY20',
       'Apr-FY20', 'May-FY20', 'Jun-FY20', 'Jul-FY20', 'Aug-FY20', 'Sept-FY20',
       'Oct-FY21', 'Nov-FY21', 'Dec-FY21', 'Jan-FY21', 'Feb-FY21', 'Mar-FY21',
       'Apr-FY21']})


dates_expanded = pd.concat([dates]*num_markets, ignore_index=True)
final_table['Month-FY'] = dates_expanded['date'].values

In [30]:
def pivot_to_array(n):
    return sales_data.iloc[(n)*num_markets : (n+1)*num_markets].sort_values(a1).drop(columns = [a1]).values.flatten()

In [31]:
#Replace with the column names you want in your final table
column_names = ['Sales', 'SSU', 'RSA', 'DSA & Reseller', 'Close Ratio', 'Sales Budget', 'Sales to Budget', 'Sales Target', 
                'Sales to Target', 'OE Call Sales', 'Net New Subs', 'Net New Budget', 'Net New to Budget', 'Net New Target', 
                'Net New to Target']


num_columns = len(column_names)
col_numbers = list(range(num_columns))

In [32]:
matrix = list(map(pivot_to_array, col_numbers))

In [33]:
for i in range(num_columns):
    final_table[column_names[i]] = matrix[i]

In [34]:
final_table

,Market,Month-FY,Sales,SSU,RSA,DSA & Reseller,Close Ratio,Sales Budget,Sales to Budget,Sales Target,Sales to Target,OE Call Sales,Net New Subs,Net New Budget,Net New to Budget,Net New Target,Net New to Target
3317,Ames,Oct FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3485,Ames,Nov FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3569,Ames,Dec FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3653,Ames,Jan FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3737,Ames,Feb FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293,Ames,Mar FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3821,Ames,Apr FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3905,Ames,May FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3989,Ames,Jun FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4073,Ames,Jul FY15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#final_table.to_csv("sales_for_power_bi.csv") #replace with what you want your new file to be called